<a href="https://colab.research.google.com/github/choarauc/form_ch/blob/main/GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 

In [2]:
link = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/CaseStudy/mobile_train.csv'

In [3]:
df = pd.read_csv(link, sep=',', encoding = "ISO-8859-1", header=0)

In [4]:
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


Les variables sont toutes numériques, bien que certaines soient des variables indicatrices.
Toutes les colonnes ont l'air correctement distribué, à priori aucune ligne n'a de valeur aberrante.

Pour la suite de l'exercice, nous allons choisir d'entraîner 3 types de modèles différents : la régression logistique, les Random Forests ainsi que les SVM.
Pour que la majorité de ces algorithmes fonctionnent, il sera nécessaire de modifier l'échelle des données, qui sont a priori assez différentes.

Mais avant d'aller plus loin, il est impératif de savoir comment évaluer un modèle.
L'évaluation d'un modèle vise à estimer l'erreur de généralisation de ce dernier, c'est-à-dire la performance du modèle choisi sur des données jamais vues auparavant.
Pour éviter une évaluation trop optimiste et un surapprentissage, il est préférable d'éviter d'estimer l'erreur de généralisation du modèle sélectionné sur les mêmes données que celles que nous avons utilisées pour sélectionner le modèle parmi d'autres.

Si la quantité de données disponibles est importante, la stratégie la plus efficace est de diviser les données en plusieurs échantillons, chacune servant un objectif particulier.

Par exemple, pour le réglage des hyperparamètres d'un modèle particulier, nous pouvons diviser aléatoirement les données en trois ensembles : train / validation / test. L'ensemble d'apprentissage est utilisé pour entraîner autant de modèles qu'il y a de combinaisons différentes d'hyperparamètres du modèle. Ces modèles sont ensuite évalués sur l'ensemble de validation, et le modèle ayant la meilleure performance sur cet ensemble de validation est sélectionné comme modèle gagnant.
Par la suite, le modèle est formé à nouveau sur l'ensemble d'apprentissage + données de validation en utilisant l'ensemble d'hyperparamètres choisi et la performance de généralisation est estimée en utilisant l'ensemble de test. Si cette erreur de généralisation est similaire à l'erreur de validation, nous avons des raisons de croire que le modèle fonctionnera bien sur des données nouvelles.

+    Lorsqu'il s'agit de données chronologiques où la tâche consiste à faire des prévisions, les ensembles d'entraînement, de validation et de test doivent être sélectionnés en séparant les données le long de l'axe temporel. C'est-à-dire que les données les plus anciennes sont utilisées pour l'entraînement, les plus récentes pour la validation et les dernières chronologiquement pour les tests.

+  Pour assurer la fiabilité de cette méthode, l'ensemble de validation doit être suffisamment grand pour pouvoir comparer les performances des modèles, de manière robuste.

> Si nous avons affaire à un jeu de données de petite taille, la stratégie diffère. L'évaluation du modèle ne change pas, nous avons toujours besoin d'un ensemble de tests sur lequel nous pouvons estimer l'erreur de généralisation du modèle final sélectionné. Nous diviserons donc les données en un ensemble d'entraînement et un ensemble de test.

# 1. Sélection d'hyperparamètres

> Pour la comparaison et la sélection des hyperparamètres, la validation croisée est le meilleur outil. Pour rappel, la validation croisée fonctionne ainsi :
Nous divisons l'ensemble d'entraînement en K ensembles plus petits.
A chaque étape, un des K échantillons est mis de côté, et l'on entraîne autant de modèles qu'il y a de combinaisons différentes d'hyperparamètres sur les K-1 échantillons restants. Le score des modèles crées sont ensuite calculés sur l'échantillon mis de côté.
Pour chaque ensemble d'hyperparamètres, nous calculons le score moyen et sélectionnons donc l'ensemble d'hyperparamètres ayant la meilleure performance moyenne sur l'ensemble des échantillons.

Cette méthode est réalisable grâce à la classe **GridSearchCV**, déjà vue dans le module précédent.

Ensuite, le modèle est réentraîné avec les hyperparamètres sélectionnés sur l'ensemble d'entraînement complet, et l'erreur de généralisation est estimée à l'aide de l'ensemble de test.

+ (a) Importer les fonctions GridSearchCV, train_test_split et cross_val_score du module sklearn.model_selection.
+ (b) Importer MinMaxScaler de sklearn.preprocessing.
+ (c) Importer la classe LogisticRegression du module sklearn.linear_model.
+ (d) Importer la classe RandomForestClassifier du module sklearn.ensemble.
+ (e) Importer la classe SVC du module sklearn.svm.

In [5]:
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

+ (f) Séparer dans deux DataFrames les variables explicatives et la variable cible.
+ (g) Créer un ensemble d'entraînement et un ensemble de tests, représentants respectivement 75% et 25% des données originales.
+ (h) Normaliser les données explicatives dans les deux ensembles, grâce à une transformation Min-Max calculée sur les données d'entraînement.

In [7]:
X, y = df.drop('price_range', axis=1), df['price_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

+ (i) Créer un classifieur clf_lr, de la classe LogisticRegression, avec le paramètre max_iter=1000.
+ (j) Créer un dictionnaire params_lr avec les hyperparamètres 'solver': ['liblinear', 'lbfgs'], et 'C': [10**(i) for i in range(-4, 3)].
+ (k) Instancier une classe gridcv de GridSearchCV, avec clf_lr, les paramètres de params_lr.

In [8]:
clf_lr = LogisticRegression(max_iter=1000)
params_lr = {'solver': ['liblinear', 'lbfgs'], 'C': [10**(i) for i in range(-4, 3)]}

gridcv = GridSearchCV(clf_lr, param_grid=params_lr, scoring='accuracy', cv=3)    

+ (l) Entraîner à présent gridcv sur l'ensemble d'entraînement.
+ (m) Afficher les résultats du GridSearch grâce à l'attribut cv_results_, dont on se contentera d'afficher les colonnes 'params', 'mean_test_score' et 'std_test_score'.

In [9]:
clf_lr = LogisticRegression(max_iter=1000)
params_lr = {'solver': ['liblinear', 'lbfgs'], 'C': [10**(i) for i in range(-4, 3)]}

gridcv = GridSearchCV(clf_lr, param_grid=params_lr, scoring='accuracy', cv=3)    

+ (l) Entraîner à présent gridcv sur l'ensemble d'entraînement.
+ (m) Afficher les résultats du GridSearch grâce à l'attribut cv_results_, dont on se contentera d'afficher les colonnes 'params', 'mean_test_score' et 'std_test_score'.

In [10]:
gridcv.fit(X_train, y_train)

pd.DataFrame(gridcv.cv_results_)[['params', 'mean_test_score', 'std_test_score']]

,params,mean_test_score,std_test_score
0,"{'C': 0.0001, 'solver': 'liblinear'}",0.398000,0.047861
1,"{'C': 0.0001, 'solver': 'lbfgs'}",0.260000,0.000000
2,"{'C': 0.001, 'solver': 'liblinear'}",0.502667,0.013888
3,"{'C': 0.001, 'solver': 'lbfgs'}",0.369333,0.019482
4,"{'C': 0.01, 'solver': 'liblinear'}",0.572000,0.015578
5,"{'C': 0.01, 'solver': 'lbfgs'}",0.616667,0.020155
6,"{'C': 0.1, 'solver': 'liblinear'}",0.664667,0.015521
7,"{'C': 0.1, 'solver': 'lbfgs'}",0.738000,0.011431
8,"{'C': 1, 'solver': 'liblinear'}",0.752667,0.003399
9,"{'C': 1, 'solver': 'lbfgs'}",0.872000,0.011431


Les résultats affichés peuvent nous permettre de choisir l'ensemble d'hyperparamètres ayant obtenu le meilleur score :
    
    Il s'agit de {'C': 100, 'solver': 'lbfgs'}.